Nom :

Prénom :

# Examen de TP d'Informatique Quantique - Consignes

Le TP est calibré pour durer 1h30 (+ 30 minutes pour les tiers-temps). Les exercices sont tous indépendants et proposés en ordre croissant de difficulté. A la fin des 1h30, vous avez 10 minutes pour m'envoyer votre notebook par mail, renommé avec votre nom.

L'examen se fait seul, aucune collaboration n'est autorisée. Vous avez cependant le droit d'utiliser vos anciens TP ainsi qu'internet. L'IA est très peu recommandée pour ce genre de sujet, et si vous décidez de l'utiliser, faites le intelligemment.

**Code à lancer pour que le TP fonctionne**


In [1]:
# Exécuter seulement dans Google Colab
!pip install myqlm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.2/833.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.5/648.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.0/661.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.0/911.0 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 742.5/742.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
   ━━━━

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from qat.lang import QRoutine, H, CNOT, RY, Z, X, CCNOT, Program
from qat.synthopline.linear_synthesis import extract_linear_operator
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

def circuit_to_map(circuit):
    n = circuit.arity
    N = 1 << n
    coords = []
    for i in range(N):
        rout = QRoutine()
        qreg = rout.new_wires(n)
        for j in range(n):
            if i & (1 << j):
                X(qreg[j])
        circuit(qreg)
        result = qpu.submit(rout.to_job())
        for sample in result:
            if sample.amplitude != 0:
                coords.append((i, sample.state.int))

    # Lis l'image du plan vierge
    img = mpimg.imread("img/plan.png")
    # Crée le plot
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img, extent=[0, 16, 0, 16])

    # Dessiner les croix
    for (x, y) in coords:
        ax.scatter(x + 0.5, 15 - y + 0.5,
                marker='x',
                color="#E67E22",
                s=200,
                linewidth=3)

    ax.set_xticks(range(17))
    ax.set_yticks(range(17))
    ax.set_title("Plan du Louvre")
    plt.gca().invert_yaxis()

    plt.show()

def plot_states(circuit):
    fig, axes = plt.subplots(2, 2, figsize=(7, 7))
    axes_flat = axes.flatten()
    for i in range(4):
        ax = axes_flat[i]
        rout = QRoutine()
        q = rout.new_wires(4)
        for j in range(2):
            if i & (1<<j):
                X(q[j])
        circuit(q)
        result = qpu.submit(rout.to_job())
        amplitude = np.zeros((16))
        for sample in result:
            amplitude[sample.state.int] = sample.probability
        ax.bar(range(16), amplitude, width=0.4, color="#95A5A6")
        ax.set_ylim(0, 0.7)
    plt.show()




# Sujet : Opération "Voler les bijoux de la Couronne"


<table>
  <tr>
    <td width="20%">
        <img src="https://github.com/oceko/QC_polytech_exam/blob/main/img/jerry.jpg?raw=1" width="100%">
    </td>
    <td width="80%">
    <h3>Briefing de Jerry</h3>
    Bonjour, nous avons une situation de la plus haute importance. Le budget du WOOHP a subi quelques coupes sombres récemment, et il est temps de récupérer un petit retour sur investissements. Notre cible : le Louvre. La mission est simple : "Voler les bijoux de la couronne".
    <center>
    <img src="https://github.com/oceko/QC_polytech_exam/blob/main/img/bijoux.jpg?raw=1" width="400">
    </center>
    Le musée est protégé par un système de sécurité quantique dernier cri. Si vous essayez de forcer les verrous de manière classique, vous déclencherez une alarme avant même d'avoir franchi le premier couloir. Gladys a modifié vos terminaux pour qu'ils puissent s'interfacer avec le processeur quantique central du Louvre.

Voici le déroulé de la mission :
1) Repérer sur le plan les bijoux à voler et les systèmes de sécurité physiques au sein du musée.
2) S'infiltrer par la porte sécurisée
3) Voler les bijoux en 7 minutes chrono
    </td>
  </tr>
</table>



## Exercice 1 : Etude du plan du musée (4 points / 20)

Comme le Louvre est un musée très sécurisé, toutes les informations dont nous avons besoin pour mettre en place le plan sont encodées. L'encodage choisi est un peu particulier, ce sont des circuits quantiques. Par chance, nous avons en notre possession un code qui permet de décoder ces circuits quantiques et obtenir l'information cachée.


Tout d'abord, il faut localiser les cibles à voler. Nous disposons du plan vierge du musée

![Plan](https://github.com/oceko/QC_polytech_exam/blob/main/img/plan.png?raw=1)

Nous avons obtenus en amont la localisation des bijoux et joyaux de la Couronne de France, mais celles-ci sont encodées via le circuit quantique suivant

![Circuit localisation](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_2.png?raw=1)

**Question 1** Implémenter le circuit quantique ci-dessus à l'aide d'une routine quantique. Pour afficher l'emplacement des différents objets à voler (il y en à 16 au total), utiliser la fonction *circuit_to_map(circuit)*.

In [6]:
def localistation():
  rout = QRoutine()
  qubits = rout.new_wires(4)
  CNOT(qubits[2],qubits[0])
  X(qubits[3])
  X(qubits[0])
  X(qubits[1])
  CCNOT(qubits[0],qubits[2],qubits[1])
  CNOT(qubits[0],qubits[1])
  CNOT(qubits[1],qubits[0])
  return rout

localistation().display()

Malheureusement, grand musée implique gros système de surveillance. Par chance, nous avons accès à la carte de l'installation de ces systèmes via le circuit quantique suivant

![Circuit sécurité](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_1.png?raw=1)

**Question 2** Implémenter le circuit quantique ci-dessus à l'aide d'une routine quantique. Pour afficher la localisation des différents systèmes de sécurité, utiliser la fonction *circuit_to_map(circuit)*.

In [11]:
def surveillance():
  rout = QRoutine()
  qubits = rout.new_wires(4)
  X(qubits[0])
  H(qubits[3])
  CNOT(qubits[0],qubits[2])
  X(qubits[3])
  H(qubits[0])
  H(qubits[1])
  rout.apply(X.ctrl(3), qubits[0], qubits[1], qubits[2], qubits[3])
  return rout

surveillance().display()

Les systèmes de sécurité présents dans le musée sont trop complexes pour que nous puissons les contourner, nous devons donc nous restreindre aux objets n'étant pas directement surveillés.

**Question 3** En regroupant les informations des questions 1 et 2, combien d'objets allons nous pouvoir dérober ?

L'image ne s'affiche pas

## Exercice 2 : Passer la porte d'entrée sécurisée (7 points / 20)

Si le Louvres était un petit musée de quartier, on pourrait imaginer passer par une fenêtre pour rentrer, mais ce n'est pas le cas. Nous devons rentrer par la porte sécurisée. Elle demande de résoudre une tâche très complexe pour un cambrioleur : une addition **AVEC** retenue. Comme vous n'avez pas pensé à prendre votre calculatrice, plus qu'une seule solution s'offre à vous : utiliser l'ordinateur quantique pour résoudre ce problème. Considérons deux entiers encodés sur 3 bits

$$
    a = a_2a_1a_0 \textrm{ et } b = b_2b_1b_0
$$

Nous souhaitons calculer la somme exacte de ces deux entiers et stocker cette valeur, qui sera encodée sur 4 bits pour prendre en considération la retenue. Pour ce faire, nous allons avoir besoin de deux opérateurs, que nous allons nommer $O_1$ et $O_2$ par manque d'originalité.

### Opérateur $O_1$
L'opérateur $O_1$ est un opérateur sur 3 qubits ayant pour circuit quantique

![Circuit O1](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_O1.png?raw=1)

**Question 1** Implémenter l'opérateur O1 correspondant au circuit quantique ci-dessus

In [15]:
def O1():
  rout = QRoutine()
  qubits = rout.new_wires(3)
  CNOT(qubits[2],qubits[1])
  CNOT(qubits[2],qubits[0])
  CCNOT(qubits[0],qubits[1],qubits[2])
  return rout

O1().display()

Supposons que le système en entrée de $O_1$ soit $|q_0\rangle|q_1\rangle|q_2\rangle$

**Question 2** Expliciter l'état en sortie à la main (en utilisant les opérateurs binaires tels que le ou exclusif "^")

Soit nos états de sortie |q0⟩|q1⟩|q2⟩, si :
- |q0⟩|q1⟩|q2⟩ = |000> => |000>
- |q0⟩|q1⟩|q2⟩ = |001> => |110>
- |q0⟩|q1⟩|q2⟩ = |010> => |010>
- |q0⟩|q1⟩|q2⟩ = |011> => |101>
- |q0⟩|q1⟩|q2⟩ = |100> => |100>
- |q0⟩|q1⟩|q2⟩ = |101> => |011>
- |q0⟩|q1⟩|q2⟩ = |110> => |111>
- |q0⟩|q1⟩|q2⟩ = |111> => |001>

D'ou
- |q0>' = |q0> xor |q2>
- |q1>' = |q1> xor |q2>
- |q2>' = MAJ(q0,q1,q2) = (q0 && q1)⊕(q1 && q2)⊕(q0 && q2)

Vérifier numériquement en affichant chaque entrée possible sur les 3 qubits $|q_0\rangle|q_1\rangle|q_2\rangle$. Vous pouvez utiliser la fonction *display_result(circuit)*.

In [29]:

def test(circuit_func, nqbits):
    for i in range(2**nqbits):
        bits_x = [int(b) for b in bin(i)[2:].zfill(nqbits)]

        rout = QRoutine()
        qbits = rout.new_wires(nqbits)

        for j in range(nqbits):
            if bits_x[j] == 1:
                rout.apply(X, qbits[j])

        rout.apply(circuit_func(), qbits)

        print(f"Résultat pour l'entrée |{''.join(map(str, bits_x))}> :")
        display_result(rout.to_circ())
        print('-' * 20)

test(O1, 3)

Résultat pour l'entrée |000> :
Etat |000>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |001> :
Etat |110>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |010> :
Etat |010>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |011> :
Etat |101>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |100> :
Etat |100>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |101> :
Etat |011>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |110> :
Etat |111>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |111> :
Etat |001>: probabilité 1.0, amplitude (1+0j)
--------------------


### Opérateur $O_2$

L'opérateur $O_2$ est décrit par le circuit suivant

![Circuit O2](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_O2.png?raw=1)

**Question 3** Implémenter la routine quantique qui décrit le circuit $O_2$


In [22]:
def O2():
  rout = QRoutine()
  qubits = rout.new_wires(3)
  CCNOT(qubits[0],qubits[1],qubits[2])
  CNOT(qubits[2],qubits[0])
  CNOT(qubits[0],qubits[1])
  return rout

O2().display()

### Additionneur

Maintenant, grâce à nos deux opérateur $O_1$ et $O_2$, nous sommes en mesure de faire une addition. Il suffit de faire une combinaison astucieuse de ces deux opératuers pour obtenir le résultat. Toujours avec nos entiers $a=a_2a_1a_0$ et $b=b_2b_1b_0$, nous allons considérer un vecteur $c$ tel que
- $c_0 = 0$, car on commence toujours un calcul sans retenue,
- $c_{i+1} = a_i \oplus b_i \oplus c_i$, où $\oplus$ représente l'opérateur "ou exclusif".

**Question 4** Vérifier sur toutes les entrées possibles sur 3 qubits, qu'en sorti de ce circuit $c_i$ et $a_i$ restent inchangés

![circuit 0102](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_O1O2.png?raw=1)

In [31]:
def sub_cirtcuit():
  rout = QRoutine()
  qubits = rout.new_wires(3)
  rout.apply(O1(), qubits)
  rout.apply(O2(), qubits)
  return rout

test(sub_cirtcuit,3)
print("on remarque a_i et b_i inchanges")


Résultat pour l'entrée |000> :
Etat |000>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |001> :
Etat |011>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |010> :
Etat |010>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |011> :
Etat |001>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |100> :
Etat |110>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |101> :
Etat |101>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |110> :
Etat |100>: probabilité 1.0, amplitude (1+0j)
--------------------
Résultat pour l'entrée |111> :
Etat |111>: probabilité 1.0, amplitude (1+0j)
--------------------
on remarque a_i et b_i inchanges


Que vaut le qubit $b_i$ en sortie ?


En sortie, $b_i$ vaut q0 $\oplus$ q1 $\oplus$ q2

Maintenant que nous avons ce petit bloc de calcul, nous pouvons construire l'additioneur au complet pour des entiers sur 3 qubits. Voici le circuit quantique correspondant

![circuit adder](https://github.com/oceko/QC_polytech_exam/blob/main/img/circuit_adder.png?raw=1)

**Question 5** Implémenter le circuit de l'additionneur donné ci-dessus

In [33]:
def additionneur():
  rout = QRoutine()
  qubits = rout.new_wires(8)
  rout.apply(O1(), qubits[0:3])
  rout.apply(O1(), qubits[3:6])
  rout.apply(O1(), qubits[4:7])
  CNOT(qubits[6],qubits[7])
  rout.apply(O2(), qubits[4:7])
  rout.apply(O2(), qubits[3:6])
  rout.apply(O2(), qubits[0:3])
  return rout

additionneur().display()


C'est le moment fatidique de savoir si votre additionneur fonctionne, et ne déclenche pas l'alarme. Sur l'écran de la porte s'affiche une série de calculs

**Question 6** Combien font $2+4$ (avec l'ordinateur quantique) ? Vous pouvez sélectionner que les qubits intéressant dans *display_result(circuit, idx=[liste des indices])*

Combien font $5 + 7$ (idem)

## Exercice 3 : Voler les bijoux (9 points / 20)

C'est bon vous avez réussi à vous infiltrer dans le Louvre, mais vous n'avez que 7 minutes pour voler les bijoux de la Couronne. Cela peut paraître court, mais nous avons trouvé un moyen de déverouiller les vitrines où ils sont exposés simultanément. Dans le musée ultra sécurisé, il y a 4 systèmes de verrou différents $i$, chacun s'ouvrant après l'application d'une unitaire $U_i$ avec une amplitude $\alpha_i$. Si nous arrivons à appliquer simultanément toutes ces unitaires alors nous pourrons nous emparer du butin en un rien de temps.

### Création de la superposition des $\alpha_i$
Dans un premier temps, vous allez devoir créer la superposition des amplitudes. Je vous les transmet ci dessous.

In [ ]:
alphas = np.array([0.1, 0.4, 0.7, 0.2])
alphas /= np.linalg.norm(alphas)
print(alphas)

[0.11952286 0.47809144 0.83666003 0.23904572]


Vous allez devoir encoder ces différentes amplitudes dans les amplitudes d'un registre quantique à 2 qubits $|\psi\rangle$ tel que

$$
|\psi\rangle = \alpha_0 |0\rangle_{a_0}|0\rangle_{a_1} + \alpha_1 |1\rangle_{a_0}|0\rangle_{a_1} + \alpha_2 |0\rangle_{a_0}|1\rangle_{a_1} + \alpha_3 |1\rangle_{a_0}|1\rangle_{a_1}.
$$

Pour ce faire, nous allons utiliser des rotations $RY(\theta)$. Pour rappel, la porte $RY(\theta)$ fonctionne de la manière suivante lorsqu'appliquée à l'état $|0\rangle$
$$
RY(\theta)|0\rangle = \cos(\theta / 2) |0\rangle + \sin(\theta / 2) |1\rangle
$$

Nous devons donc déterminer en amont 3 probabilités différentes, desquelles découleront des angles de rotation pour nos RY

1) $p_1$ qui est la probabilité sur le premier qubit $a_0$ à 0
2) $p_2$ qui est la probabilité sur le deuxième qubit $a_1$ à 0, sachant que le premier est à 0
3) $p_3$ qui est la probabilité sur le deuxième qubit $a_1$ à 0, sachant que le premier est à 1

**Question 1** Trouver ces probabilités à partir des amplitudes données par le vecteur *alphas*


Maintenant, il faut passer des probabilités aux angles de rotation pour les RY associées, en utilisant notamment la fonction arccos.

**Question 2** Exprimer les angles $\theta_1$, $\theta_2$ et $\theta_3$ tels que $RY(\theta_i) |0\rangle = \sqrt{p_i} |0\rangle + \dots$

**Question 3** Construire le circuit quantique qui :
1) Crée la superposition $\sqrt{p_1} |0\rangle + \sqrt{1-p_1}|1\rangle$ sur le qubit $a_0$
2) Crée la superposition $\sqrt{p_2} |0\rangle + \sqrt{1-p_2}|1\rangle$ sur le qubit $a_1$, contrôlée par le qubit $a_0$ à $0$ (en utilisant une RY contrôlée encadrée de 2 portes X)
3) Crée la superposition $\sqrt{p_3} |0\rangle + \sqrt{1-p_3}|1\rangle$ sur le qubit $a_1$, contrôlée par le qubit $a_0$ à $1$ (en utilisant une RY contrôlée)

On peut alors vérifier que la bonne superposition des amplitudes est créée en appelant *display_result* sur ce circuit

### Création des unitaires $U_i$

Maintenant que vous avez la superposition des $\alpha_i$, occupons nous des 4 unitaires $U_i$. Ce sont des unitaires agissant sur 2 qubits et décrites chacune par un circuit quantique en particulier.

1) $U_0$ : opérateur identité $\rightarrow$ Circuit sans porte quantique
2) $U_1$ donné en image ci-dessous
3) $U_2$ donné en image ci-dessous
4) $U_4$ : circuit permettant de réaliser une paire de Bell (description donnée ci-dessous)

Le circuit de $U_0$ étant trivial, son implémentation est déjà donnée.

In [ ]:
def U_0():
    rout = QRoutine()
    qreg = rout.new_wires(2)

    return rout

#### Circuit $U_1$

Voici le circuit correspondant à l'unitaire $U_1$

![circuit U1](https://github.com/oceko/QC_polytech_exam/blob/main/img/ex3_circuit2.png?raw=1)

**Question 4** Implémenter le circuit correspondant

#### Circuit $U_2$

Voici le circuit correspondant à l'unitaire $U_2$
![U2](https://github.com/oceko/QC_polytech_exam/blob/main/img/ex3_circuit3.png?raw=1)

**Question 5** Implémenter le circuit correspondant

#### Circuit $U_3$

Le dernier circuit n'est pas donné directement mais nous avons obtenu l'information qu'il permet de créer une paire de Bell suivant l'opération suivante :

$$
U_3 |0\rangle_{q_0}|0\rangle_{q_1} = \frac{1}{\sqrt{2}} (|0\rangle_{q_0}|1\rangle_{q_1} + |1\rangle_{q_0}|0\rangle_{q_1})
$$

**Question 6** Implémenter le circuit quantique correspondant à cette opération

### Combinaison des unitaires

Maintenant, il ne reste plus qu'à tout combiner ensemble. L'objectif est de créer l'unitaire suivante

$$
\sum_i \alpha_i U_i,
$$

qui représente donc la superposition des unitaires $U_i$ que nous avons pu implémenter juste avant, suivant la distribution de probabilité donnée par le vecteur contenant les $\alpha_i$.

L'algorithme de combinaison se décompose en 3 étapes et utilise 4 qubits : 2 qubits de donnée $q_0$ et $q_1$ sur lesquels nous allons appliquer nos opérateurs $U_i$ et 2 qubits ancillaires de travail $a_0$ et $a_1$ qui vont servir à créer la superposition des unitaires. Voici les étapes à suivre :
1) Préparer la superposition $|\psi\rangle = \alpha_0 |0\rangle_{a_0}|0\rangle_{a_1} + \alpha_1 |1\rangle_{a_0}|0\rangle_{a_1} + \alpha_2 |0\rangle_{a_0}|1\rangle_{a_1} + \alpha_3 |1\rangle_{a_0}|1\rangle_{a_1}$.
2) Appliquer les différentes unitaires $U_i$ sur les qubits $q_0$ et $q_1$, en les controllant par les qubits ancillaires $a_0a_1$ dans l'état $|i\rangle$
3) Défaire la superposition d'état sur les qubits $a_0$ et $a_1$

Pour simplifier le travail, une représentation sous forme de circuit est donnée ci-dessous

![LCU](https://github.com/oceko/QC_polytech_exam/blob/main/img/lcu.png?raw=1)

**Question 7** Implémenter ce circuit quantique

Utilisez la fonction *plot_states(circuit)* pour afficher la réponse du système à votre circuit.

Voici la réponse que nous sommes censés obtenir pour les 4 types d'entrée différentes possibles.

![Final output](https://github.com/oceko/QC_polytech_exam/blob/main/img/final_outpu.png?raw=1)